<a href="https://colab.research.google.com/github/bineee39/first-repository/blob/master/%EC%9E%91%EC%82%AC%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow

print(tensorflow.__version__)

2.19.0


## 가사는 kaggle 가사 데이터를 가져왔습니다.


In [ ]:
import os
import re
import zipfile
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. 파일 경로 설정 (코랩 로컬 파일 시스템 기준)
# 현재 이미지를 보면 파일이 루트 경로에 위치한 것으로 보입니다.
zip_path = './archive (7).zip'

# 압축 해제할 임시 디렉토리 설정
extract_dir = './extracted_lyrics'
os.makedirs(extract_dir, exist_ok=True)
print(f" 설정 완료. 압축 파일 경로: {zip_path}")

 설정 완료. 압축 파일 경로: ./archive (7).zip


In [ ]:
# 2. ZIP 파일 압축 해제
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        print(f"🔍 {zip_path} 압축 해제 중...")
        zip_ref.extractall(extract_dir)
    print("압축 해제 완료!")
except FileNotFoundError:
    print(f"오류: 지정된 경로에 파일이 없습니다. 경로를 확인해 주세요: {zip_path}")
    raw_corpus = []
else:
    # 3. 압축 해제된 디렉토리에서 모든 CSV 파일 읽기
    csv_files = glob.glob(os.path.join(extract_dir, '**', '*.csv'), recursive=True)
    raw_corpus = []

    print(f"{len(csv_files)}개의 CSV 파일에서 데이터 읽기 시작...")

    TEXT_COLUMN_NAME = 'Lyric'

    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)

            # 가사 텍스트 컬럼이 데이터프레임에 있는지 확인
            if TEXT_COLUMN_NAME not in df.columns:
                 print(f"경고: {csv_file}에 '{TEXT_COLUMN_NAME}' 컬럼이 없습니다. 첫 번째 컬럼을 사용합니다.")
                 text_column = df.columns[0]
            else:
                 text_column = TEXT_COLUMN_NAME

            # 텍스트가 NaN이 아닌 행만 필터링하여 리스트에 추가
            lines = df[text_column].dropna().astype(str).tolist()
            raw_corpus.extend(lines)

        except Exception as e:
            print(f"경고: {csv_file} 처리 중 오류 발생 - {e}")

    # 모든 리스트 요소가 문장 단위로 저장되도록 분리 및 빈 줄 제거
    final_corpus = []
    for line in raw_corpus:
        final_corpus.extend(line.split('\n'))

    raw_corpus = [line.strip() for line in final_corpus if line.strip()]

    print(f"데이터 크기: {len(raw_corpus)} 문장")
    print("Examples:\n", raw_corpus[:3])

🔍 ./archive (7).zip 압축 해제 중...
압축 해제 완료!
21개의 CSV 파일에서 데이터 읽기 시작...
데이터 크기: 5981 문장
Examples:
 ["rihanna work work work work work work he said me haffi work work work work work work he see me do mi dirt dirt dirt dirt dirt dirt so me put in work work work work work work when you ah guh learn learn learn learn learn meh nuh cyar if him hurt hurt hurt hurt hurting   rihanna dry me a desert him nuh time to have you lurking him ah go act like he nuh like it you know i dealt with you the nicest nuh body touch me you nuh righteous nuh badda text me in a crisis i believed all of your dreams adoration you took my heart and my keys and my patience you took my heart on my sleeve for decoration you mistaken my love i brought for you for foundation all that i wanted from you was to give me something that i never had something that you've never seen something that you've never been mmmmm but i wake up and act like nothing's wrong just get ready fi   rihanna work work work work work work he said me 

In [ ]:
# 4. 문장 정제 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

# 정제 함수 적용
corpus = list(map(preprocess_sentence, raw_corpus))
print(f"정제된 데이터 크기: {len(corpus)}")

# 5. 토큰화 및 패딩 함수
def tokenize(corpus, num_words, maxlen):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words,
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    sequences = tokenizer.texts_to_sequences(corpus)

    # Filter out sequences longer than maxlen
    filtered_sequences = [seq for seq in sequences if len(seq) <= maxlen]

    tensor = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, padding='post', maxlen=maxlen)
    return tensor, tokenizer, filtered_sequences # Return filtered_sequences as well

# 하이퍼파라미터 설정
NUM_WORDS = 12000 # 사용할 단어 개수
MAX_LEN = 15      # 문장 최대 길이

tensor, tokenizer, filtered_sequences = tokenize(corpus, NUM_WORDS, MAX_LEN)
print(f"토큰화 후 길이 {MAX_LEN} 이하인 문장 수: {len(filtered_sequences)} 문장 (필터링 전: {len(corpus)} 문장)")
print(f"텐서 변환 완료. 텐서 모양: {tensor.shape}")

정제된 데이터 크기: 5981
토큰화 후 길이 15 이하인 문장 수: 114 문장 (필터링 전: 5981 문장)
텐서 변환 완료. 텐서 모양: (114, 15)


In [ ]:
# 6. Source (입력)와 Target (정답) 분리
enc_inputs = tensor[:, :-1]
dec_targets = tensor[:, 1:]

# 7. 훈련 데이터와 검증 데이터 분리 (80% 훈련, 20% 검증)
enc_train, enc_val, dec_train, dec_val = train_test_split(
    enc_inputs,
    dec_targets,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# 8. tf.data.Dataset 구성
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 64 # 코랩 메모리를 고려한 배치 사이즈

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=False)

print("\n데이터셋 준비 완료.")


데이터셋 준비 완료.


In [29]:
import tensorflow as tf

# 9. 모델 정의
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out

# 하이퍼파라미터 설정
VOCAB_SIZE = tokenizer.num_words + 1
EMBEDDING_SIZE = 256
HIDDEN_SIZE = 256

model = TextGenerator(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE)

# 10. 모델 컴파일 및 학습
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

# Calculate steps per epoch based on estimated time (approx 5 minutes per epoch)
# Original: 27727 steps took 1022s -> ~36.8ms/step
# For 300s (5 minutes) / 0.0368s/step = ~8152 steps. Let's use 8139 for safety margin.
TRAIN_STEPS_PER_EPOCH = 8139

# Calculate validation steps (validation data size: len(enc_val) = 23, BATCH_SIZE = 64)
# ceil(23 / 64) = 1
VALIDATION_STEPS = 1

print(f"\n모델 학습 시작. (Epochs=5, steps_per_epoch={TRAIN_STEPS_PER_EPOCH})")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    steps_per_epoch=TRAIN_STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS
)


모델 학습 시작. (Epochs=5, steps_per_epoch=8139)
Epoch 1/5
8139/8139 ━━━━━━━━━━━━━━━━━━━━ 301s 37ms/step - loss: 1.0982 - val_loss: 5.6645
Epoch 2/5
8139/8139 ━━━━━━━━━━━━━━━━━━━━ 299s 37ms/step - loss: 0.2909 - val_loss: 7.0445
Epoch 3/5
8139/8139 ━━━━━━━━━━━━━━━━━━━━ 299s 37ms/step - loss: 0.2904 - val_loss: 7.4892
Epoch 4/5
8139/8139 ━━━━━━━━━━━━━━━━━━━━ 302s 37ms/step - loss: 0.2902 - val_loss: 8.0588
Epoch 5/5
8139/8139 ━━━━━━━━━━━━━━━━━━━━ 304s 37ms/step - loss: 0.2901 - val_loss: 8.3346


In [36]:
import tensorflow as tf
import numpy as np # tf.random.categorical 사용을 위해 numpy 필요

# 11. 문장 생성 함수 (Sampling 적용)
def generate_text_sampling(model, tokenizer, init_sentence="<start>", max_len=60, temperature=1.0):

    # [입력 문장 전처리 부분은 이전과 동일]
    if not init_sentence.startswith('<start>'):
        if ' <end>' in init_sentence:
            init_sentence = init_sentence.split(' <end>')[0]
        if not init_sentence.startswith('<start>'):
            init_sentence = '<start> ' + init_sentence

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index.get("<end>")

    while True:
        predict = model(test_tensor)

        # 1. Temperature 적용 (Softmax 전에 확률 분포의 경사를 완만하게 만듦)
        predict = predict / temperature

        # 2. 확률 분포 계산 (Softmax)
        predict_proba = tf.nn.softmax(predict, axis=-1)[:, -1]

        # 3. 확률 분포에 따라 단어 샘플링 (가장 높은 확률만 선택하지 않음)
        # tf.math.log를 사용하여 logit을 확률로 변환
        predict_word = tf.random.categorical(
            tf.math.log(predict_proba), num_samples=1
        )
        predict_word = tf.squeeze(predict_word, axis=1) # [1, 1] -> [1] 텐서 모양 정리

        # 4. test_tensor 업데이트 (순환)
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 종료 조건
        if end_token and predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        word = tokenizer.index_word.get(word_index, "")
        if word not in ('<start>', '<end>', '<pad>', '<unk>', ''):
            generated += word + " "

    return generated.strip()

# 12. 결과 확인 (Sampling 적용)
# temperature=1.0은 일반적인 무작위성, 0.5는 더 보수적, 1.5는 더 창의적/무작위적
TEMPERATURE = 0.8

print(f"\n🔥 모델이 생성한 가사 (Sampling 적용, Temperature={TEMPERATURE}):\n")

print("--- i wish로 시작 ---")
print(generate_text_sampling(model, tokenizer, init_sentence="i wish", max_len=60, temperature=TEMPERATURE))
print(generate_text_sampling(model, tokenizer, init_sentence="i wish", max_len=60, temperature=TEMPERATURE))
print(generate_text_sampling(model, tokenizer, init_sentence="i wish", max_len=60, temperature=TEMPERATURE))

print("\n--- i see the로 시작 ---")
print(generate_text_sampling(model, tokenizer, init_sentence="i see the", max_len=60, temperature=TEMPERATURE))
print(generate_text_sampling(model, tokenizer, init_sentence="i see the", max_len=60, temperature=TEMPERATURE))
print(generate_text_sampling(model, tokenizer, init_sentence="i see the", max_len=60, temperature=TEMPERATURE))


🔥 모델이 생성한 가사 (Sampling 적용, Temperature=0.8):

--- i wish로 시작 ---
i wish can you
i wish can you
i wish can you

--- i see the로 시작 ---
i see the full
i see the full
i see the full


.......kaggle 가사에 카디비가 있었는데 가사가 너무 선정적이라 그런지 처음에 가사 뽑았을때 이상한 가사가 나와서 코드수정했습니다..................